In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import linearmodels as lm
import matplotlib
import math
import statsmodels.formula.api as smf

from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects
from linearmodels.panel import FirstDifferenceOLS
from linearmodels.panel import compare
from matplotlib import pyplot as plt

from pandas.api.types import is_numeric_dtype

pd.set_option('display.max_columns', 500)

%matplotlib inline

In [25]:
# read the main data set
df=pd.read_csv('final_dataset.csv',
                    encoding='utf-8')


In [26]:
# separate Bus, Rail, DemandResponsive and Other
bus = df[df['RailBus']=='Bus']
rail = df[df['RailBus']=='Rail']
demand_responsive = df[df['RailBus']=='DemandResponsive']
other = df[df['RailBus']=='Other']


In [27]:
# set the indices
bus=bus.set_index(['MNAME','Year'])
rail=rail.set_index(['MNAME','Year'])

In [28]:
# keep only the numeric columns -- the estimation will give an error otherwise
bus = bus.select_dtypes(include=[np.number])
rail = rail.select_dtypes(include=[np.number])

In [29]:
# create a log of all fields
for col in bus.columns:
    bus[col+'_log'] = np.log(bus[col]+1)
    
for col in rail.columns:
    rail[col+'_log'] = np.log(rail[col]+1)    

In [31]:
# check the data
bus.head()

Unnamed: 0    Id2  Cluster_upt          upt  \
MNAME                Year                                                
Akron, OH Metro Area 2005           0  10420            1  1294595.792   
                     2006           3  10420            1  1363776.042   
                     2007           6  10420            1  1237511.271   
                     2008           9  10420            1  1246293.688   
                     2009          12  10420            1  1000725.896   

                                   VRM  Total_HH  HH_0Veh  HH_1Veh  HH_2Veh  \
MNAME                Year                                                     
Akron, OH Metro Area 2005  572693.6458    276416    20661    85198   112380   
                     2006  536282.7292    280837    19248    97567   109326   
                     2007  619889.9375    281669    21066    94071   113232   
                     2008  565919.2292    281731    19035   100229   108332   
                     2009  522729.1458    281769    23447    97811   108215   

                           HH_3Veh  HH_4+Veh  CBSA Code   Pop_TSD  \
MNAME                Year                                           
Akron, OH Metro Area 2005    40847     17330    10420.0  100521.5   
                     2006    38795     15901    10420.0   99593.0   
                     2007    36917     16383    10420.0   98664.5   
                     2008    38548     15587    10420.0   97736.0   
                     2009    36339     15957    10420.0   96807.5   

                           Tot_Emp_TSD  Tot_Retail_TSD  gasPrice  HH_MED_INC  \
MNAME                Year                                                      
Akron, OH Metro Area 2005      41364.0          4110.0  2.246667       44719   
                     2006      43378.0          3771.0  2.544250       44507   
                     2007      41173.0          3626.0  2.821583       47898   
                     2008      41542.0          3830.0  3.229500       50036   
                     2009      40430.0          3743.0  2.332250       47482   

                           HH_MEAN_INC  INC_U35  INC_35_100  INC_100P  \
MNAME                Year                                               
Akron, OH Metro Area 2005        58676     38.7        46.8      14.5   
                     2006        58859     38.8        47.2      14.0   
                     2007        63454     37.2        46.1      16.7   
                     2008        65801     36.4        45.1      18.5   
                     2009        60668     36.8        48.1      15.1   

                           Tot_Pop  Tot_Instate_Pop  Tot_Outstate_Pop  \
MNAME                Year                                               
Akron, OH Metro Area 2005   684459           521491            138184   
                     2006   700943           530777            139714   
                     2007   699356           536148            135625   
                     2008   698553           529519            137011   
                     2009   699935           537803            134499   

                           Tot_NonUSA_POP  Total_Median_Income_Individual  \
MNAME                Year                                                   
Akron, OH Metro Area 2005            2945                           24612   
                     2006            4695                           23925   
                     2007            3455                           25217   
                     2008            4452                           25357   
                     2009            2466                           24856   

                           Native_Instate_Med_Inc_Indiv  \
MNAME                Year                                 
Akron, OH Metro Area 2005                         24899   
                     2006                         24129   
                     2007                         25202   
                     2008                         

In [32]:
rail.head()

Unnamed: 0    Id2  Cluster_upt           upt  \
MNAME                Year                                                 
Akron, OH Metro Area 2005           2  10420            1  8.678297e+05   
                     2006           5  10420            1  9.114397e+05   
                     2007           8  10420            1  9.561626e+05   
                     2008          11  10420            1  1.002934e+06   
                     2009          14  10420            1  6.861524e+05   

                                   VRM  Total_HH  HH_0Veh  HH_1Veh  HH_2Veh  \
MNAME                Year                                                     
Akron, OH Metro Area 2005  211674.3333    276416    20661    85198   112380   
                     2006  219053.2500    280837    19248    97567   109326   
                     2007  209295.1667    281669    21066    94071   113232   
                     2008  203862.4167    281731    19035   100229   108332   
                     2009  191086.0000    281769    23447    97811   108215   

                           HH_3Veh  HH_4+Veh  CBSA Code   Pop_TSD  \
MNAME                Year                                           
Akron, OH Metro Area 2005    40847     17330    10420.0  100521.5   
                     2006    38795     15901    10420.0   99593.0   
                     2007    36917     16383    10420.0   98664.5   
                     2008    38548     15587    10420.0   97736.0   
                     2009    36339     15957    10420.0   96807.5   

                           Tot_Emp_TSD  Tot_Retail_TSD  gasPrice  HH_MED_INC  \
MNAME                Year                                                      
Akron, OH Metro Area 2005      41364.0          4110.0  2.246667       44719   
                     2006      43378.0          3771.0  2.544250       44507   
                     2007      41173.0          3626.0  2.821583       47898   
                     2008      41542.0          3830.0  3.229500       50036   
                     2009      40430.0          3743.0  2.332250       47482   

                           HH_MEAN_INC  INC_U35  INC_35_100  INC_100P  \
MNAME                Year                                               
Akron, OH Metro Area 2005        58676     38.7        46.8      14.5   
                     2006        58859     38.8        47.2      14.0   
                     2007        63454     37.2        46.1      16.7   
                     2008        65801     36.4        45.1      18.5   
                     2009        60668     36.8        48.1      15.1   

                           Tot_Pop  Tot_Instate_Pop  Tot_Outstate_Pop  \
MNAME                Year                                               
Akron, OH Metro Area 2005   684459           521491            138184   
                     2006   700943           530777            139714   
                     2007   699356           536148            135625   
                     2008   698553           529519            137011   
                     2009   699935           537803            134499   

                           Tot_NonUSA_POP  Total_Median_Income_Individual  \
MNAME                Year                                                   
Akron, OH Metro Area 2005            2945                           24612   
                     2006            4695                           23925   
                     2007            3455                           25217   
                     2008            4452                           25357   
                     2009            2466                           24856   

                           Native_Instate_Med_Inc_Indiv  \
MNAME                Year                                 
Akron, OH Metro Area 2005                         24899   
                     2006                         24129   
                     2007                         25202   
                     2008                  

In [40]:
# basic log model for bus, all clusters
mod=PanelOLS.from_formula('upt_log \
                    ~ VRM_log \
                    + Tot_Pop_log \
                    + Total_Labor_MSA_log \
                    + gasPrice_log \
                    + EntityEffects \
                    ',data=bus)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                upt_log   R-squared:                        0.9661
Estimator:                   PanelOLS   R-squared (Between):              0.8583
No. Observations:                1249   R-squared (Within):               0.9661
Date:                Mon, Sep 09 2019   R-squared (Overall):              0.8599
Time:                        13:49:01   Log-likelihood                   -138.53
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      8035.0
Entities:                         116   P-value                           0.0000
Avg Obs:                       10.767   Distribution:                  F(4,1129)
Min Obs:                       6.0000                                           
Max Obs:                       11.000   F-statistic (robust):             8035.0
                            

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


In [41]:
# basic log model for bus, all clusters
# use pop and emp in transit supportive density (TSD)

mod=PanelOLS.from_formula('upt_log \
                    ~ VRM_log \
                    + Pop_TSD_log \
                    + Tot_Emp_TSD_log \
                    + gasPrice_log \
                    + EntityEffects \
                    ',data=bus)
res=mod.fit()
print(res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                upt_log   R-squared:                        0.9660
Estimator:                   PanelOLS   R-squared (Between):              0.9936
No. Observations:                1684   R-squared (Within):               0.9660
Date:                Mon, Sep 09 2019   R-squared (Overall):              0.9934
Time:                        13:53:12   Log-likelihood                    17.426
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                   1.082e+04
Entities:                         156   P-value                           0.0000
Avg Obs:                       10.795   Distribution:                  F(4,1524)
Min Obs:                       6.0000                                           
Max Obs:                       11.000   F-statistic (robust):          1.082e+04
                            

C:\Program Files\Anaconda3\lib\site-packages\linearmodels\utility.py:476: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)
